In [1]:
# Install FastAPI and Uvicorn
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.0 MB/s eta 0:00:00


In [2]:
# Import necessary libraries for data analysis and visualization
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [3]:
# Create and train a DummyClassifier

_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [4]:
# Function to evaluate the model's performance on test data
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
   # testacc=accuracy_score(y_test, y_pred)
   # testrecall=recall_score(y_test, y_pred)
   # testprecision=precision_score(y_test, y_pred)
    testacc = accuracy_score(y_test, y_pred)
    testrecall = recall_score(y_test, y_pred, pos_label='True', zero_division=0)
    testprecision = precision_score(y_test, y_pred, pos_label='True', zero_division=0)
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [5]:
# Create a DataFrame for evaluation, including model predictions and correctness
 # Combine x and y for prediction
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [6]:
# Create a FastAPI app
app = fastapi.FastAPI()

# Predict endpoint - takes an id, x, and y, makes a prediction, and stores it
@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
   # X = np.rec.fromarrays([_x, _y])[:(-1*last_n)]
   # Y = np.rec.fromarrays([_result])[:(-1*last_n)]

# Select the most recent 'last_n' data points from _x, _y, and _result for retraining
    print(f"RETRAINING over last {last_n} samples")
    X = np.array(list(zip(_x[-last_n:], _y[-last_n:])))
    Y = np.array(_result[-last_n:])

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(n_neighbors=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [7]:
# Insert 50 points
for i in range(50):
    _x.append(i)
    _y.append(i)
    _result.append("True" if i % 2 == 0 else "False")

In [8]:
# Change to Dummy model and retrain using the most recent 50 samples
new_model("Dummy", 50, strategy="stratified")


Dummy 50 stratified 0
RETRAINING over last 50
RETRAINING over last 50 samples
[[ 5  5]
 [25 25]
 [23 23]
 [41 41]
 [30 30]
 [12 12]
 [43 43]
 [ 1  1]
 [48 48]
 [10 10]
 [45 45]
 [33 33]
 [44 44]
 [ 9  9]
 [ 3  3]
 [ 2  2]
 [ 8  8]
 [36 36]
 [18 18]
 [34 34]
 [39 39]
 [26 26]
 [ 0  0]
 [28 28]
 [19 19]
 [21 21]
 [24 24]
 [35 35]
 [32 32]
 [ 6  6]]
['False' 'False' 'False' 'False' 'True' 'True' 'False' 'False' 'True'
 'True' 'False' 'False' 'True' 'False' 'False' 'True' 'True' 'True' 'True'
 'True' 'False' 'True' 'True' 'True' 'False' 'False' 'True' 'False' 'True'
 'True']
Trained Model Test Data Accuracy Score : 45.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        11
        True       0.45      1.00      0.62         9

    accuracy                           0.45        20
   macro avg       0.23      0.50      0.31        20
weighted avg       0.20      0.45      0.28        20

{'message': "Model trained DummyClassifier(stra

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.45,
  'recall': 1.0,
  'precision': 0.45}}

In [9]:
# Set model to KNN with k=3 and retrain on the most recent 50 samples
new_model("KNN", 50, k=3)

KNN 50 None 3
RETRAINING over last 50
RETRAINING over last 50 samples
[[43 43]
 [36 36]
 [37 37]
 [ 6  6]
 [33 33]
 [28 28]
 [20 20]
 [42 42]
 [30 30]
 [18 18]
 [ 7  7]
 [ 2  2]
 [22 22]
 [13 13]
 [27 27]
 [21 21]
 [29 29]
 [15 15]
 [ 3  3]
 [38 38]
 [41 41]
 [46 46]
 [35 35]
 [ 1  1]
 [ 9  9]
 [31 31]
 [47 47]
 [44 44]
 [23 23]
 [26 26]]
['False' 'True' 'False' 'True' 'False' 'True' 'True' 'True' 'True' 'True'
 'False' 'True' 'True' 'False' 'False' 'False' 'False' 'False' 'False'
 'True' 'False' 'True' 'False' 'False' 'False' 'False' 'False' 'True'
 'False' 'True']
Trained Model Test Data Accuracy Score : 60.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00         8
        True       0.60      1.00      0.75        12

    accuracy                           0.60        20
   macro avg       0.30      0.50      0.38        20
weighted avg       0.36      0.60      0.45        20

{'message': 'Model trained KNeighborsClassifier(n_ne

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': 'Model trained KNeighborsClassifier(n_neighbors=3)',
 'train': {'message': 'Model retrained',
  'accuracy': 0.6,
  'recall': 1.0,
  'precision': 0.6}}

In [10]:
#print(predict(1, 1, 1))
#result(1, True)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
RETRAINING over last 50 samples
[[ 4  4]
 [27 27]
 [10 10]
 [32 32]
 [ 0  0]
 [13 13]
 [ 2  2]
 [19 19]
 [22 22]
 [48 48]
 [12 12]
 [24 24]
 [21 21]
 [18 18]
 [ 7  7]
 [35 35]
 [49 49]
 [31 31]
 [ 6  6]
 [ 3  3]
 [30 30]
 [14 14]
 [ 8  8]
 [46 46]
 [41 41]
 [23 23]
 [25 25]
 [ 9  9]
 [45 45]
 [47 47]]
['True' 'False' 'True' 'True' 'True' 'False' 'True' 'False' 'True' 'True'
 'True' 'True' 'False' 'True' 'False' 'False' 'False' 'False' 'True'
 'False' 'True' 'True' 'True' 'True' 'False' 'False' 'False' 'False'
 'False' 'False']
Trained Model Test Data Accuracy Score : 50.0
 
              precision    recall  f1-score   support

       False       0.00      0.00      0.00        10
        True       0.50      1.00      0.67        10

    accuracy                           0.50        20
   macro avg       0.25      0.50      0.33        20
weighted avg       0.25      0.50      0.33        20

{'message': "Model trained DummyClassifier(str

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.5,
  'recall': 1.0,
  'precision': 0.5}}

In [11]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [12]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())